In [ ]:
# pip install -q opencv-python
# !pip install --upgrade pyreadline
# !pip install torchsummary 
# !pip install --upgrade fastai

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import argparse

In [ ]:
import generate_opencv_gt_images

In [ ]:
from fastai.vision.all import *

In [ ]:
from torch.utils.tensorboard import SummaryWriter
torch.cuda.memory_allocated(0) / 1024.0 / 1024.

In [ ]:
generate_data = True
args = argparse.Namespace()
args.output_dir = "test1_train"
args.debug = False
args.num_drawings = 5000
args.num_images_per_drawing = 1
if generate_data:
    generate_opencv_gt_images.run(args)

In [ ]:
input_path = Path(args.output_dir)
input_path.ls()

In [ ]:
def get_dl_image_files(path):
    return [path / f.stem for f in path.glob("img-?????-???.json")]

def get_dl_target_image(fprefix: Path):
    return fprefix.with_suffix(".aliased.png")

def get_dl_source_image(fprefix: Path):
    return fprefix.with_suffix(".rendered.png")

In [ ]:
# input_prefixes = get_dl_image_files(input_path)
# images = [f.with_suffix(".rendered.png") for f in input_prefixes]
# pil_images = [PILImage.create(f) for f in images]
# show_images(pil_images)

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, ImageBlock),
                           get_items = get_dl_image_files,
                           get_y     = get_dl_target_image,
                           get_x     = get_dl_source_image,                   
                           splitter  = RandomSplitter())
dls = dblock.dataloaders(input_path, bs=6)
dsets = dblock.datasets(input_path)
dls.show_batch()

In [ ]:
mean, std = model_meta.get(models.resnet18)['stats']
mean = torch.tensor(mean).cuda()
std = torch.tensor(std).cuda()
# print (mean, std)
# dls.c = 3 # number of outputs
# dblock.summary(input_path)
def metric_l1_raw_image(x,y):
    #print (x.shape)
    #print (y.shape)
    # * std + mean
    diff = (x-y).abs()
    # print (diff)
    # print (255.0*diff)
    for c in range(0,3):
        diff[:,c,:,:] *= std[c] # FIXME there must be a way to avoid the loop
    #print (255.0*diff)
    return 255.0 * diff.mean()

In [ ]:
learn = unet_learner(dls, arch=models.resnet18, loss_func=nn.MSELoss(), n_out=3, metrics=AvgMetric(metric_l1_raw_image))

In [ ]:
#dblock.summary(input_path)

In [ ]:
# learn.load("model1.ai")

In [ ]:
# writer = SummaryWriter()
# writer.add_graph(learn.model.cpu(), first(dls.valid)[0].cpu())

In [ ]:
# learn.lr_find()

In [ ]:
print ('Occupied mem: ', torch.cuda.memory_allocated(0) / 1024.0 / 1024.)

In [ ]:
learn.lr = 2e-3
learn.fine_tune(40)

In [ ]:
b = dls.valid.one_batch()
_,_,preds = learn.get_preds(dl=[b], with_decoded=True)

In [ ]:
dls.show_batch()

In [ ]:
learn.save("model2.ai")

In [ ]:
# FIXME: why the valid metric does not match the typical values we find here of ~2

input = dls.decode_batch(dls.one_batch())[0]
test_img = input[0]
gt_img = input[1]

predicted_img = learn.predict(test_img)

display(predicted_img[0][:,0,0])
display(predicted_img[1][:,0,0])
display(predicted_img[2][:,0,0])
display(predicted_img[2][:,0,0]*255.0)
display(gt_img[:,0,0])

# FIXME: can't figure out the decoding actually applied to predicted_img. It seems off.
# predicted_img[2] is fine once normalized, predicted_ing[0] is supposed to be the
# perfect one but it's not.

# preds = learn.get_preds(dls.valid, with_input=True, with_decoded=True)
plt.imshow(test_img.permute(1, 2, 0))
plt.figure()
plt.imshow(gt_img.permute(1, 2, 0))
plt.figure()
plt.imshow(predicted_img[2].permute(1, 2, 0))
(gt_img - predicted_img[2]*255.0).abs().mean()
metric_l1_raw_image((gt_img / 255.0).unsqueeze(1).cuda(), predicted_img[2].unsqueeze(1).cuda())

In [ ]:
dls.decode((np.ones((1,3,256,256)), np.ones((1,3,256,256))))unsqueeze

In [ ]:
fname = Path("./downloaded.jpg")
if fname.exists(): os.remove(fname)
# url = "https://www.r-graph-gallery.com/img/graph/lineplot-spaghetti1.png"
url = "http://www.telesens.co/wp-content/uploads/2018/03/img_5aa5827c1d42c.png"
download_url(url, fname)
assert fname.exists()

In [ ]:
Path().resolve().parent.parent.ls()

In [ ]:
fname = Path().resolve().parent.parent / 'Plots' / 'Industry_GDP.png'
input = PILImage.create(fname)
show_images([input, learn.predict(input)[2]], figsize=(64,64))

In [ ]:
class HistogramPerLabel:
    def __init__(self, im, labels):
        self.histogram = {}
        hsv_im = cv2.cvtColor(im, cv2.COLOR_RGB2HSV)
        for l in labels:
            self.histogram[l] = compute_histogram_for_label(l, hsv_im, labels)
            
    def compute_histogram_for_label(l, hsv_im, labels):
        numpy.histogram()

In [ ]:
histogram[label][hsv] -> count

In [ ]:
def metric_segmentation_accuracy_from_color_thresholding(rgb_im, labels):
    